In [ ]:
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/kshida/learning-ml/blob/main/rinna-gpt-tutorial.ipynb)

# 自然言語処理に触れてみよう！

自然言語処理用のライブラリをインストールします。  
- [Transformers](https://github.com/huggingface/transformers)：アメリカのHugging Face社が提供している自然言語処理用のライブラリ。Hugging Faceは[機械学習用のサイト](https://huggingface.co/)も公開している（日本語はないけれど・・）
- [Sentencepiece](https://github.com/google/sentencepiece)：Google先生作成の言語をセンテンスごとに分割してくれるトークナイザ。形態素解析に比べて、遥かに少ない語彙数で学習できる

In [ ]:
!pip install transformers
!pip install sentencepiece

続いて、rinna社が[先日提供開始](https://rinna.co.jp/%E3%83%8B%E3%83%A5%E3%83%BC%E3%82%B9/f/%E6%97%A5%E6%9C%AC%E8%AA%9E%E3%81%AB%E7%89%B9%E5%8C%96%E3%81%97%E3%81%9F13%E5%84%84%E3%83%91%E3%83%A9%E3%83%A1%E3%83%BC%E3%82%BF%E3%81%AEgpt%E8%A8%80%E8%AA%9E%E3%83%A2%E3%83%87%E3%83%AB%E3%82%92%E5%85%AC%E9%96%8B)した日本語GPTモデルを読み込みます。  
※ GPTは、単語の確率の組み合わせから次の単語の確率を計算する言語モデルのこと。

In [ ]:
import torch
from transformers import T5Tokenizer, AutoModelForCausalLM

# トークナイザとモデル本体を読み込む
tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-gpt-1b")
model = AutoModelForCausalLM.from_pretrained("rinna/japanese-gpt-1b")

# GPU使用時の設定
if torch.cuda.is_available():
    model = model.to("cuda")

それでは文章を生成してみましょう！

In [ ]:
# 入力文（任意の言葉を入れる）
text = "1on1で重要なことは"

# 入力文のトークナイズ
token_ids = tokenizer.encode(text, add_special_tokens=False, return_tensors="pt")

# 入力文とパラメータの設定にしたがって文章を生成
with torch.no_grad():
    output_ids = model.generate(
        token_ids.to(model.device),
        max_length=100,
        min_length=100,
        do_sample=True,
        top_k=500,
        top_p=0.95,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        bad_word_ids=[[tokenizer.unk_token_id]]
    )

# 生成された文章を単語IDから実際の単語に変換
output = tokenizer.decode(output_ids.tolist()[0])
print(output)